In [74]:
from urllib.request import Request,urlopen
from lxml import html
import pandas as pd
import requests

# My Code (still in progress)

In [75]:
# URL we want parsed
product = "B07DWBLGV9"
url = "https://www.amazon.com/dp/" + product

In [80]:
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
# Creating request package
req = requests.get(url,headers=header)
if req.status_code==404:
    print({"url":amazon_url,"error":"page not found"})
#if req.status_code!=200:

In [92]:
# Open the url
html_chunks = req.text.split('<div id="Ask">')
data = {}
#openlink = urlopen(req)
#parsed_page = html.parse(openlink)
# Close the Browser
#openlink.close()

In [106]:
for chunks in html_chunks:
    parser = html.fromstring(chunks)
    XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
    print(XPATH_REVIEW_SECTION_1)
    reviews = parser.xpath(XPATH_REVIEW_SECTION_1)
    reviews_list = []
    for review in reviews:
        XPATH_REVIEW_TEXT_1 = './/div[@data-hook="review-collapsed"]//text()'
        raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_1)
        review_text = ' '.join(' '.join(raw_review_text1).split())
        reviews_list.append(review_text)
#print(reviews_list)

//div[contains(@id,"reviews-summary")]
//div[contains(@id,"reviews-summary")]


In [64]:
#results = root.xpath("//t[@id = '%s']" % toner_id)
extracted_review = parsed_page.xpath(".//div[@data-hook = 'review-collapsed']//text()" )
# Decrypt the extracted_reviews
plain_review = []
for x in extracted_review:
    decrypted_review = x.text_content()
    plain_review.append(decrypted_review)


In [65]:
Data = pd.DataFrame(plain_review)
print(Data)

Empty DataFrame
Columns: []
Index: []


# Code I found Online

## https://www.scrapehero.com/how-to-scrape-amazon-product-reviews/

In [4]:
from lxml import html  
import json
import requests
import json,re
from dateutil import parser as dateparser
from time import sleep

def ParseReviews(asin):
    #This script has only been tested with Amazon.com
    amazon_url  = 'http://www.amazon.com/dp/'+asin
    # Add some recent user agent to prevent amazon from blocking the request 
    # Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
    
    for i in range(5):  
        response = requests.get(amazon_url,headers = headers,verify=False)	
        if response.status_code==404:
            return {"url":amazon_url,"error":"page not found"}
        if response.status_code!=200:
            continue
            
        '''
        The HTML xpath parser from lxml.etree seems to have max depth limit. 
        It won't traverse further to parse the text if the depth exceeds 254. 
        To avoid this, splitting the html into chunks
        '''
        html_chunks = response.text.split('<div id="Ask">')
        
        data = {}
        for chunks in html_chunks:
            parser = html.fromstring(chunks)
            XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
            XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
            XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'
            
            XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
            XPATH_PRODUCT_NAME = '//h1//span[@id="productTitle"]//text()'
            XPATH_PRODUCT_PRICE  = '//span[@id="priceblock_ourprice"]/text()'
            
            raw_product_price = parser.xpath(XPATH_PRODUCT_PRICE)
            raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
            
            product_price = ''.join(raw_product_price).replace(',','')
            product_name = ''.join(raw_product_name).strip()

            data.update({"name":product_name}) if product_name else data.get("name")
            data.update({"price":product_price}) if product_price else data.get("price")

            total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
            reviews = parser.xpath(XPATH_REVIEW_SECTION_1)
            
            if not reviews:
                reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
            ratings_dict = {}
            reviews_list = []

            #grabing the rating  section in product page
            for ratings in total_ratings:
                extracted_rating = ratings.xpath('./td//a//text()')
                if extracted_rating:
                    rating_key = extracted_rating[0] 
                    raw_raing_value = extracted_rating[1]
                    rating_value = raw_raing_value
                    if rating_key:
                        ratings_dict.update({rating_key:rating_value})   
                        
            #Parsing individual reviews
            for review in reviews:
                XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
                XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//text()'
                XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]//text()'
                XPATH_REVIEW_TEXT_1 = './/div[@data-hook="review-collapsed"]//text()'
                XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
                XPATH_REVIEW_COMMENTS = './/span[@data-hook="review-comment"]//text()'
                XPATH_AUTHOR  = './/span[contains(@class,"profile-name")]//text()'
                XPATH_REVIEW_TEXT_3  = './/div[contains(@id,"dpReviews")]/div/text()'
                
                raw_review_author = review.xpath(XPATH_AUTHOR)
                raw_review_rating = review.xpath(XPATH_RATING)
                raw_review_header = review.xpath(XPATH_REVIEW_HEADER)
                raw_review_posted_date = review.xpath(XPATH_REVIEW_POSTED_DATE)
                raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_1)
                raw_review_text2 = review.xpath(XPATH_REVIEW_TEXT_2)
                raw_review_text3 = review.xpath(XPATH_REVIEW_TEXT_3)

                #cleaning data
                author = ' '.join(' '.join(raw_review_author).split())
                review_rating = ''.join(raw_review_rating).replace('out of 5 stars','')
                review_header = ' '.join(' '.join(raw_review_header).split())

                try:
                    review_posted_date = dateparser.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
                except:
                    review_posted_date = None
                review_text = ' '.join(' '.join(raw_review_text1).split())

                #grabbing hidden comments if present
                if raw_review_text2:
                    json_loaded_review_data = json.loads(raw_review_text2[0])
                    json_loaded_review_data_text = json_loaded_review_data['rest']
                    cleaned_json_loaded_review_data_text = re.sub('<.*?>','',json_loaded_review_data_text)
                    full_review_text = review_text+cleaned_json_loaded_review_data_text
                else:
                    full_review_text = review_text
                if not raw_review_text1:
                    full_review_text = ' '.join(' '.join(raw_review_text3).split())
                    
                raw_review_comments = review.xpath(XPATH_REVIEW_COMMENTS)
                review_comments = ''.join(raw_review_comments)
                review_comments = re.sub('[A-Za-z]','',review_comments).strip()
                review_dict = {
                                'review_comment_count':review_comments,
                                'review_text':full_review_text,
                                'review_posted_date':review_posted_date,
                                'review_header':review_header,
                                'review_rating':review_rating,
                                'review_author':author

                            }
                reviews_list.append(review_dict)

            data.update({
                        'ratings':ratings_dict,
                        'reviews':reviews_list,
                        'url':amazon_url,
                
                    })
        return data

    return {"error":"failed to process the page","url":amazon_url}

def ReadAsin():
    #Add your own ASINs here 
    AsinList = ['B07DWBLGV9','B017HW9DEW',"B00U8KSIOM"]
    extracted_data = []
    for asin in AsinList:
        print("Downloading and processing page http://www.amazon.com/dp/"+asin)
        extracted_data.append(ParseReviews(asin))
        sleep(5)
    f = open('data.json','w')
    json.dump(extracted_data,f,indent=4,ensure_ascii=False)

if __name__ == '__main__':
    ReadAsin()

c:\users\yuxia\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


c:\users\yuxia\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


c:\users\yuxia\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [8]:
f = open("data.json", 'r')
data = json.loads(f.read())

In [20]:
review_data = pd.concat([pd.DataFrame(data[i]['reviews']) for i in range(3)], ignore_index=True)

In [21]:
import pandas as pd

In [22]:
review_data

,review_author,review_comment_count,review_header,review_posted_date,review_rating,review_text
0,Jimmy,,nice projector,24 Jul 2018,5.0,"This projector is great, is easy to connect wi..."
1,roca,,"it is a good projector, but u need to buy spar...",02 Aug 2018,5.0,"it is a good projector, but u need to buy spar..."
2,Frank,,Great Value,14 Aug 2018,5.0,Image quality is better when dark in the room....
3,Peanut,,We got it to work.,25 Sep 2018,5.0,Ot does what it says it is going to do. It wou...
4,Frankie D.,,No one beat the price,29 Jul 2018,5.0,This is a first projector that I’ve been waiti...
5,Alex,,Easy to use,18 Aug 2018,5.0,"This is second projector I have, really nice l..."
6,nancy,,Good price,11 Aug 2018,5.0,"I use this for my son’s party,a five years old..."
7,Jasson,,Five stars,18 Jul 2018,5.0,It’s really great to watch movies with it at n...
8,Veronica Hale,,Very good for Virgin Mobile.,25 Apr 2016,5.0,This phone is one of the best I have had from ...
9,DM,,Best budget phone hands down!,09 Jul 2016,5.0,This phone is the best virgin mobile phone I h...
